<a href="https://colab.research.google.com/github/rishabhk0/end_to_end_ml/blob/Drowsiness_Detection_OpenCV/RealTime_Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js


def use_webcam():
  display(Javascript('''
    let videoStream = null;

    async function setupCamera() {
      const video = document.createElement('video');
      video.autoplay = true;
      video.playsInline = true;
      video.id = 'webcamVideo';
      document.body.appendChild(video);

      const constraints = { video: true };
      const stream = await navigator.mediaDevices.getUserMedia(constraints);
      video.srcObject = stream;
      videoStream = stream;
      await video.play();
      return video;
    }

    async function captureFrame() {
      let video = document.getElementById('webcamVideo');
      if (!video) {
        video = await setupCamera();
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      const context = canvas.getContext('2d');
      context.drawImage(video, 0, 0, canvas.width, canvas.height);
      return canvas.toDataURL('image/jpeg');
    }

    function stopCamera() {
      if (videoStream) {
        videoStream.getTracks().forEach(track => track.stop());
        videoStream = null;
      }
      const video = document.getElementById('webcamVideo');
      if (video) {
        video.srcObject = null;
        video.remove();
      }
      console.log('Camera stopped.');
    }

    window.captureFrame = captureFrame;
    window.stopCamera = stopCamera;
  '''))

def get_frame():
  js_code = 'window.captureFrame()'
  data_url = eval_js(js_code)

  binary_data = b64decode(data_url.split(',')[1])
  np_array = np.frombuffer(binary_data, np.uint8)
  frame = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
  return frame


use_webcam()

model = load_model('best_drowsiness_model.h5')

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_eye.xml'
)

IMG_SIZE = 24
score = 0
threshold = 15



print("Please grant webcam permissions when prompted by your browser.")

while True:
    frame = get_frame()
    if frame is None:
        print("Error: Failed to grab frame from webcam.")

        continue

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    eye_detected_in_frame = False

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        if len(eyes) > 0:
            eye_detected_in_frame = True

        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

            eye_img = roi_color[ey:ey+eh, ex: ex+ew]
            eye_img = cv2.resize(eye_img, (IMG_SIZE, IMG_SIZE))
            eye_img = eye_img / 255.0
            eye_img = np.expand_dims(eye_img, axis=0)

            prediction = model.predict(eye_img, verbose=0)

            if prediction[0][0] > 0.5:
                label = "Closed"
                score += 1
            else:
                label = "Open"
                score = 0

            cv2.putText(frame, label, (x + ex, y + ey - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                        (0, 255, 0), 2)

    if not eye_detected_in_frame and score > 0:
        score = 0

    if score > threshold:
        cv2.putText(frame, "DROWSY ALERT!", (100, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5,
                    (0, 0, 255), 3)

    cv2_imshow(frame)


